# Module 04 - Notebook 02: Hugging Face Inference SDK

## Learning Objectives
- Use Hugging Face Inference API for model inference
- Compare inference performance across models
- Handle rate limits and errors
- Understand pricing and quotas

---

## 1. Setup

In [ ]:
!pip install -q huggingface-hub python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
import time

load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

# Initialize inference client
client = InferenceClient(token=HF_TOKEN)
print("✓ Inference client ready")

## 2. Basic Text Generation

In [ ]:
# Simple text generation
prompt = "Write a haiku about artificial intelligence."

response = client.text_generation(
    prompt,
    model="mistralai/Mistral-7B-Instruct-v0.3",
    max_new_tokens=100,
    temperature=0.7
)

print("Prompt:", prompt)
print("\nResponse:")
print(response)

## 3. Chat Completion

In [ ]:
# Chat-based interaction
messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": "Write a Python function to check if a number is prime."}
]

response = client.chat_completion(
    messages=messages,
    model="mistralai/Mistral-7B-Instruct-v0.3",
    max_tokens=500
)

print("Assistant:")
print(response.choices[0].message.content)

## 4. Streaming Responses

In [ ]:
# Stream the response token by token
prompt = "Explain quantum computing in simple terms."

print("Streaming response:")
for token in client.text_generation(
    prompt,
    model="mistralai/Mistral-7B-Instruct-v0.3",
    max_new_tokens=200,
    stream=True
):
    print(token, end="", flush=True)
print("\n\n✓ Streaming complete")

## 5. Model Comparison

In [ ]:
# Compare different models on the same prompt
prompt = "What are the three laws of robotics?"

models = [
    "mistralai/Mistral-7B-Instruct-v0.3",
    "microsoft/Phi-3-mini-4k-instruct",
]

for model in models:
    print(f"\n{'='*60}")
    print(f"Model: {model}")
    print(f"{'='*60}")
    
    start = time.time()
    try:
        response = client.text_generation(
            prompt,
            model=model,
            max_new_tokens=150
        )
        elapsed = time.time() - start
        
        print(response)
        print(f"\n⏱️ Time: {elapsed:.2f}s")
    except Exception as e:
        print(f"Error: {e}")

## 6. Error Handling and Rate Limits

In [ ]:
from huggingface_hub import HfHubHTTPError
import time

def safe_inference(prompt: str, model: str, max_retries: int = 3):
    """Inference with retry logic for rate limits."""
    
    for attempt in range(max_retries):
        try:
            response = client.text_generation(
                prompt,
                model=model,
                max_new_tokens=100
            )
            return response
            
        except HfHubHTTPError as e:
            if "rate limit" in str(e).lower():
                wait_time = 2 ** attempt  # Exponential backoff
                print(f"⚠️ Rate limited. Waiting {wait_time}s...")
                time.sleep(wait_time)
            else:
                raise
    
    raise Exception("Max retries exceeded")

# Test the function
result = safe_inference(
    "What is machine learning?",
    "mistralai/Mistral-7B-Instruct-v0.3"
)
print(result)

## 7. Task-Specific APIs

In [ ]:
# Summarization
long_text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, 
in contrast to the natural intelligence displayed by humans and animals. 
Leading AI textbooks define the field as the study of "intelligent agents": 
any device that perceives its environment and takes actions that maximize 
its chance of successfully achieving its goals. Colloquially, the term 
"artificial intelligence" is often used to describe machines that mimic 
"cognitive" functions that humans associate with the human mind, such as 
"learning" and "problem solving".
"""

summary = client.summarization(long_text)
print("Summary:")
print(summary)

## Exercise: Build a Model Testing Harness

Create a function that:
1. Tests multiple models with the same prompt
2. Measures response time for each
3. Compares output quality
4. Handles errors gracefully

In [ ]:
# TODO: Complete this exercise
def test_models(prompt: str, models: list) -> dict:
    """Test multiple models and return comparison."""
    results = {}
    
    # Your code here
    
    return results

# Test your function
# results = test_models(
#     "Explain neural networks",
#     ["mistralai/Mistral-7B-Instruct-v0.3", "microsoft/Phi-3-mini-4k-instruct"]
# )

## Summary

You learned:
- ✅ Basic inference with Hugging Face API
- ✅ Chat completions and streaming
- ✅ Comparing models
- ✅ Error handling and rate limits

## Next Steps
- 📘 Notebook 03: Ollama Local Deployment